# hotel

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

# 데이터 로드
df = pd.read_csv("/Users/hyunwoo/Desktop/PADA/PADA_LAB/raw/hotel.csv")

# 텍스트 데이터를 전처리
df['Review_Text'] = df['Review_Text'].fillna('')

# 텍스트 데이터를 TF-IDF 벡터로 변환
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df['Review_Text'])

# NMF 모델을 사용하여 주제 모델링 수행
num_topics = 5  # 원하는 주제의 개수 설정
nmf = NMF(n_components=num_topics, random_state=42)
nmf.fit(X)

# 각 주제에 대한 단어 목록 출력
terms = np.array(vectorizer.get_feature_names_out())
for topic_idx, topic in enumerate(nmf.components_):
    print(f"Topic {topic_idx + 1}:")
    print(" ".join([terms[i] for i in topic.argsort()[:-11:-1]]))
    print()

# 주제기여도를 0이면 아주 작은 값으로 대체 (0.00000000000000001)
topic_contributions = nmf.transform(X)
topic_contributions = np.maximum(topic_contributions, 1e-20)  # 0을 매우 작은 값으로 대체

# 'depth' 열 추가: log10을 씌운 주제기여도 합산
depth = np.sum(np.log10(topic_contributions), axis=1)

# 주제 기여도의 합을 1로 정규화
topic_contributions_normalized = topic_contributions / topic_contributions.sum(axis=1)[:, np.newaxis]

# KL Divergence 계산 함수 (Breadth 계산)
def calculate_kl_divergence(review_topic_proportions, global_topic_proportions):
    # 0이 발생하지 않도록 작은 값 추가
    review_topic_proportions = review_topic_proportions + 1e-10
    global_topic_proportions = global_topic_proportions + 1e-10
    
    # KL 발산 계산
    kl_divergence = np.sum(review_topic_proportions * np.log2(review_topic_proportions / global_topic_proportions))
    
    # NaN 또는 Inf 값이 발생할 경우 0으로 처리
    if np.isnan(kl_divergence) or np.isinf(kl_divergence):
        return np.nan  # NaN이나 Inf일 경우 NaN 반환
    else:
        return kl_divergence

# 전체 문서에 대한 주제 비율의 평균 계산 (global_topic_proportions)
global_topic_proportions = np.mean(topic_contributions_normalized, axis=0)

# 각 문서에 대해 KL 발산(Breadth) 계산
breadth = np.apply_along_axis(lambda x: calculate_kl_divergence(x, global_topic_proportions), axis=1, arr=topic_contributions_normalized)

# DataFrame에 depth와 breadth 추가
df['depth'] = np.sum(np.log10(topic_contributions), axis=1)
df['breadth'] = breadth

# 주제 기여도 정규화된 값도 DataFrame에 추가
for i in range(num_topics):
    df[f"Topic_{i+1}"] = topic_contributions_normalized[:, i]

# 정규화된 결과를 CSV로 저장
df.to_csv('hotel_reviews_with_topics_and_depth_breadth.csv', index=False)

print("Topic distribution with depth and breadth added to 'hotel_reviews_with_topics_and_depth_breadth.csv'")


Topic 1:
room small nice bed hotel clean bathroom comfortable didnt shower

Topic 2:
location perfect excellent central amazing cleanliness rooms fantastic ok price

Topic 3:
staff friendly helpful excellent clean lovely hotel comfortable rooms breakfast

Topic 4:
good breakfast value food money hotel bit choice nice beds

Topic 5:
great value clean comfortable lovely customer attentive stay money rooms

Topic distribution with depth and breadth added to 'hotel_reviews_with_topics_and_depth_breadth.csv'


# amazon

In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

# 데이터 로드
df = pd.read_csv("/Users/hyunwoo/Desktop/PADA/PADA_LAB/raw/amazon.csv")

# 텍스트 데이터를 전처리
df['Review_Text'] = df['Review_Text'].fillna('')

# 텍스트 데이터를 TF-IDF 벡터로 변환
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df['Review_Text'])

# NMF 모델을 사용하여 주제 모델링 수행
num_topics = 5  # 원하는 주제의 개수 설정
nmf = NMF(n_components=num_topics, random_state=42)
nmf.fit(X)

# 각 주제에 대한 단어 목록 출력
terms = np.array(vectorizer.get_feature_names_out())
for topic_idx, topic in enumerate(nmf.components_):
    print(f"Topic {topic_idx + 1}:")
    print(" ".join([terms[i] for i in topic.argsort()[:-11:-1]]))
    print()

# 주제기여도를 0이면 아주 작은 값으로 대체 (0.00000000000000001)
topic_contributions = nmf.transform(X)
topic_contributions = np.maximum(topic_contributions, 1e-20)  # 0을 매우 작은 값으로 대체

# 'depth' 열 추가: log10을 씌운 주제기여도 합산
depth = np.sum(np.log10(topic_contributions), axis=1)

# 주제 기여도의 합을 1로 정규화
topic_contributions_normalized = topic_contributions / topic_contributions.sum(axis=1)[:, np.newaxis]

# KL Divergence 계산 함수 (Breadth 계산)
def calculate_kl_divergence(review_topic_proportions, global_topic_proportions):
    # 0이 발생하지 않도록 작은 값 추가
    review_topic_proportions = review_topic_proportions + 1e-10
    global_topic_proportions = global_topic_proportions + 1e-10
    
    # KL 발산 계산
    kl_divergence = np.sum(review_topic_proportions * np.log2(review_topic_proportions / global_topic_proportions))
    
    # NaN 또는 Inf 값이 발생할 경우 0으로 처리
    if np.isnan(kl_divergence) or np.isinf(kl_divergence):
        return np.nan  # NaN이나 Inf일 경우 NaN 반환
    else:
        return kl_divergence

# 전체 문서에 대한 주제 비율의 평균 계산 (global_topic_proportions)
global_topic_proportions = np.mean(topic_contributions_normalized, axis=0)

# 각 문서에 대해 KL 발산(Breadth) 계산
breadth = np.apply_along_axis(lambda x: calculate_kl_divergence(x, global_topic_proportions), axis=1, arr=topic_contributions_normalized)

# DataFrame에 depth와 breadth 추가
df['depth'] = np.sum(np.log10(topic_contributions), axis=1)
df['breadth'] = breadth

# 주제 기여도 정규화된 값도 DataFrame에 추가
for i in range(num_topics):
    df[f"Topic_{i+1}"] = topic_contributions_normalized[:, i]

# 정규화된 결과를 CSV로 저장
df.to_csv('amazon_reviews_with_topics_and_depth_breadth.csv', index=False)

print("Topic distribution with depth and breadth added to 'amazon_reviews_with_topics_and_depth_breadth.csv'")


Topic 1:
great product price sound quality value worked buy sounds fast

Topic 2:
good quality product price sound value far really pretty money

Topic 3:
works perfectly expected fine advertised perfect great described just exactly

Topic 4:
work love like just sound mouse tv use bought quality

Topic 5:
easy install use set mount tv sturdy setup super wall

Topic distribution with depth and breadth added to 'amazon_reviews_with_topics_and_depth_breadth.csv'


# audible

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

# 데이터 로드
df = pd.read_csv("/Users/hyunwoo/Desktop/PADA/PADA_LAB/raw/audible.csv")

# 텍스트 데이터를 전처리
df['Review_Text'] = df['Review_Text'].fillna('')

# 텍스트 데이터를 TF-IDF 벡터로 변환
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df['Review_Text'])

# NMF 모델을 사용하여 주제 모델링 수행
num_topics = 5  # 원하는 주제의 개수 설정
nmf = NMF(n_components=num_topics, random_state=42)
nmf.fit(X)

# 각 주제에 대한 단어 목록 출력
terms = np.array(vectorizer.get_feature_names_out())
for topic_idx, topic in enumerate(nmf.components_):
    print(f"Topic {topic_idx + 1}:")
    print(" ".join([terms[i] for i in topic.argsort()[:-11:-1]]))
    print()

# 주제기여도를 0이면 아주 작은 값으로 대체 (0.00000000000000001)
topic_contributions = nmf.transform(X)
topic_contributions = np.maximum(topic_contributions, 1e-20)  # 0을 매우 작은 값으로 대체

# 'depth' 열 추가: log10을 씌운 주제기여도 합산
depth = np.sum(np.log10(topic_contributions), axis=1)

# 주제 기여도의 합을 1로 정규화
topic_contributions_normalized = topic_contributions / topic_contributions.sum(axis=1)[:, np.newaxis]

# KL Divergence 계산 함수 (Breadth 계산)
def calculate_kl_divergence(review_topic_proportions, global_topic_proportions):
    # 0이 발생하지 않도록 작은 값 추가
    review_topic_proportions = review_topic_proportions + 1e-10
    global_topic_proportions = global_topic_proportions + 1e-10
    
    # KL 발산 계산
    kl_divergence = np.sum(review_topic_proportions * np.log2(review_topic_proportions / global_topic_proportions))
    
    # NaN 또는 Inf 값이 발생할 경우 0으로 처리
    if np.isnan(kl_divergence) or np.isinf(kl_divergence):
        return np.nan  # NaN이나 Inf일 경우 NaN 반환
    else:
        return kl_divergence

# 전체 문서에 대한 주제 비율의 평균 계산 (global_topic_proportions)
global_topic_proportions = np.mean(topic_contributions_normalized, axis=0)

# 각 문서에 대해 KL 발산(Breadth) 계산
breadth = np.apply_along_axis(lambda x: calculate_kl_divergence(x, global_topic_proportions), axis=1, arr=topic_contributions_normalized)

# DataFrame에 depth와 breadth 추가
df['depth'] = np.sum(np.log10(topic_contributions), axis=1)
df['breadth'] = breadth

# 주제 기여도 정규화된 값도 DataFrame에 추가
for i in range(num_topics):
    df[f"Topic_{i+1}"] = topic_contributions_normalized[:, i]

# 정규화된 결과를 CSV로 저장
df.to_csv('audible_reviews_with_topics_and_depth_breadth.csv', index=False)

print("Topic distribution with depth and breadth added to 'amazon_reviews_with_topics_and_depth_breadth.csv'")


Topic 1:
book recommend like just love listening really life highly good

Topic 2:
story characters love really good enjoyed life like narration narrator

Topic 3:
great listen job narrator narration performance does did fun recommend

Topic 4:
loved absolutely listening minute recommend wait characters narration amazing narrator

Topic 5:
read ive books best time listened listen written years times

Topic distribution with depth and breadth added to 'amazon_reviews_with_topics_and_depth_breadth.csv'


# coursera

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

# 데이터 로드
df = pd.read_csv("/Users/hyunwoo/Desktop/PADA/PADA_LAB/raw/coursera.csv", encoding = 'cp949')

# 텍스트 데이터를 전처리
df['Review_Text'] = df['Review_Text'].fillna('')

# 텍스트 데이터를 TF-IDF 벡터로 변환
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df['Review_Text'])

# NMF 모델을 사용하여 주제 모델링 수행
num_topics = 5  # 원하는 주제의 개수 설정
nmf = NMF(n_components=num_topics, random_state=42)
nmf.fit(X)

# 각 주제에 대한 단어 목록 출력
terms = np.array(vectorizer.get_feature_names_out())
for topic_idx, topic in enumerate(nmf.components_):
    print(f"Topic {topic_idx + 1}:")
    print(" ".join([terms[i] for i in topic.argsort()[:-11:-1]]))
    print()

# 주제기여도를 0이면 아주 작은 값으로 대체 (0.00000000000000001)
topic_contributions = nmf.transform(X)
topic_contributions = np.maximum(topic_contributions, 1e-20)  # 0을 매우 작은 값으로 대체

# 'depth' 열 추가: log10을 씌운 주제기여도 합산
depth = np.sum(np.log10(topic_contributions), axis=1)

# 주제 기여도의 합을 1로 정규화
topic_contributions_normalized = topic_contributions / topic_contributions.sum(axis=1)[:, np.newaxis]

# KL Divergence 계산 함수 (Breadth 계산)
def calculate_kl_divergence(review_topic_proportions, global_topic_proportions):
    # 0이 발생하지 않도록 작은 값 추가
    review_topic_proportions = review_topic_proportions + 1e-10
    global_topic_proportions = global_topic_proportions + 1e-10
    
    # KL 발산 계산
    kl_divergence = np.sum(review_topic_proportions * np.log2(review_topic_proportions / global_topic_proportions))
    
    # NaN 또는 Inf 값이 발생할 경우 0으로 처리
    if np.isnan(kl_divergence) or np.isinf(kl_divergence):
        return np.nan  # NaN이나 Inf일 경우 NaN 반환
    else:
        return kl_divergence

# 전체 문서에 대한 주제 비율의 평균 계산 (global_topic_proportions)
global_topic_proportions = np.mean(topic_contributions_normalized, axis=0)

# 각 문서에 대해 KL 발산(Breadth) 계산
breadth = np.apply_along_axis(lambda x: calculate_kl_divergence(x, global_topic_proportions), axis=1, arr=topic_contributions_normalized)

# DataFrame에 depth와 breadth 추가
df['depth'] = np.sum(np.log10(topic_contributions), axis=1)
df['breadth'] = breadth

# 주제 기여도 정규화된 값도 DataFrame에 추가
for i in range(num_topics):
    df[f"Topic_{i+1}"] = topic_contributions_normalized[:, i]

# 정규화된 결과를 CSV로 저장
df.to_csv('coursera_reviews_with_topics_and_depth_breadth.csv', index=False)

print("Topic distribution with depth and breadth added to 'coursera_reviews_with_topics_and_depth_breadth.csv'")


Topic 1:
good course beginners experience introduction content start explanation learning teaching

Topic 2:
great course experience learning introduction content teacher instructor start intro

Topic 3:
course really amazing best thank learn lot python awesome programming

Topic 4:
excellent course teaching content instructor teacher introduction beginners material explanation

Topic 5:
nice course experience introduction explanation teaching content learning beginners teacher

Topic distribution with depth and breadth added to 'coursera_reviews_with_topics_and_depth_breadth.csv'
